## RiskGrid - Automated Data Diagnostic Script
### Extracts all necessary information about your dataset

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

# CONFIGURATION

In [ ]:
DATA_PATH = r"A:\Work\RiskGrid\data\raw\chicago_crimes.csv"
SAMPLE_SIZE = None

print("=" * 80)
print(" " * 25 + "RISKGRID DATA DIAGNOSTIC")
print("=" * 80)
print(f"\nAnalyzing: {DATA_PATH}")
print(f"Sample size: {SAMPLE_SIZE if SAMPLE_SIZE else 'Full dataset'}")
print("-" * 80)

# 1. LOAD DATA

In [ ]:
print("\n[1/7] Loading data...")
try:
    df = pd.read_csv(DATA_PATH, nrows=SAMPLE_SIZE, low_memory=False)
    print(f"✓ Loaded {len(df):,} rows")
except Exception as e:
    print(f"✗ Error loading data: {e}")
    exit(1)


# 2. COLUMN IDENTIFICATION

In [ ]:
print("\n[2/7] Identifying key columns...")
print("-" * 80)

all_columns = df.columns.tolist()
print(f"Total columns: {len(all_columns)}")
print("\nAll column names:")
for i, col in enumerate(all_columns, 1):
    print(f"  {i:2d}. {col}")

# Auto-detect key columns
date_col = None
lat_col = None
lon_col = None
crime_type_col = None

# Find date column
date_keywords = ['date', 'time', 'datetime', 'occurred', 'reported']
for col in df.columns:
    if any(keyword in col.lower() for keyword in date_keywords):
        date_col = col
        break

# Find latitude column
lat_keywords = ['lat', 'latitude', 'y']
for col in df.columns:
    if any(keyword in col.lower() for keyword in lat_keywords):
        if 'location' not in col.lower():
            lat_col = col
            break

# Find longitude column
lon_keywords = ['lon', 'long', 'longitude', 'x']
for col in df.columns:
    if any(keyword in col.lower() for keyword in lon_keywords):
        if 'location' not in col.lower():
            lon_col = col
            break

# Find crime type column
type_keywords = ['type', 'category', 'primary', 'description', 'offense']
for col in df.columns:
    if any(keyword in col.lower() for keyword in type_keywords):
        if df[col].dtype == 'object':  # Should be text
            crime_type_col = col
            break

print("\n" + "=" * 80)
print("KEY COLUMNS DETECTED:")
print("=" * 80)
print(f"  Date/Time Column : {date_col if date_col else '❌ NOT FOUND'}")
print(f"  Latitude Column  : {lat_col if lat_col else '❌ NOT FOUND'}")
print(f"  Longitude Column : {lon_col if lon_col else '❌ NOT FOUND'}")
print(f"  Crime Type Column: {crime_type_col if crime_type_col else '❌ NOT FOUND'}")
print("=" * 80)

# 3. DATA QUALITY ANALYSIS

In [ ]:
print("\n[3/7] Data quality check...")
print("-" * 80)

print(f"Total rows: {len(df):,}")
print(f"\nDataset shape: {df.shape[0]:,} rows × {df.shape[1]} columns")

print("\n\nMISSING VALUES:")
print("-" * 80)
missing_summary = []
for col in df.columns:
    missing_count = df[col].isnull().sum()
    if missing_count > 0:
        missing_pct = (missing_count / len(df)) * 100
        missing_summary.append({
            'Column': col,
            'Missing': missing_count,
            'Percentage': missing_pct
        })

if missing_summary:
    missing_df = pd.DataFrame(missing_summary).sort_values('Percentage', ascending=False)
    print(missing_df.to_string(index=False))
else:
    print("✓ No missing values found!")

# Check coordinate quality
if lat_col and lon_col:
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    
    valid_coords = df[[lat_col, lon_col]].dropna()
    coord_coverage = len(valid_coords) / len(df) * 100
    
    print(f"\n\nCOORDINATE COVERAGE:")
    print("-" * 80)
    print(f"Valid coordinates: {len(valid_coords):,} / {len(df):,} ({coord_coverage:.2f}%)")
    
    if len(valid_coords) > 0:
        print(f"\nCoordinate ranges:")
        print(f"  Latitude : {valid_coords[lat_col].min():.6f} to {valid_coords[lat_col].max():.6f}")
        print(f"  Longitude: {valid_coords[lon_col].min():.6f} to {valid_coords[lon_col].max():.6f}")

# 4. DATE RANGE ANALYSIS

In [ ]:
print("\n\n[4/7] Date/time analysis...")
print("-" * 80)

if date_col:
    try:
        df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
        valid_dates = df[date_col].dropna()
        
        print(f"Date column: '{date_col}'")
        print(f"Valid dates: {len(valid_dates):,} / {len(df):,} ({len(valid_dates)/len(df)*100:.2f}%)")
        
        if len(valid_dates) > 0:
            print(f"\nDATE RANGE:")
            print(f"  Earliest: {valid_dates.min()}")
            print(f"  Latest  : {valid_dates.max()}")
            
            time_span = (valid_dates.max() - valid_dates.min()).days
            print(f"  Time span: {time_span:,} days ({time_span/365.25:.1f} years)")
            
            # Extract temporal features
            df['year'] = df[date_col].dt.year
            df['month'] = df[date_col].dt.month
            df['hour'] = df[date_col].dt.hour
            df['dayofweek'] = df[date_col].dt.dayofweek
            
            print(f"\nYears covered: {sorted(df['year'].dropna().unique().astype(int).tolist())}")
            print(f"Total unique dates: {df[date_col].dt.date.nunique():,}")
    except Exception as e:
        print(f"✗ Error parsing dates: {e}")
else:
    print("✗ No date column detected")

# 5. CRIME PATTERNS

In [ ]:
# Load just 10 rows
df = pd.read_csv(DATA_PATH, nrows=10)

print("Columns found:")
print(df.columns.tolist())

print("\nFirst row:")
print(df.iloc[0])

print("\nPrimary Type values:")
print(df['Primary Type'].tolist())

In [ ]:
print("\n\n[5/7] Crime pattern analysis...")
print("-" * 80)

if crime_type_col:
    print(f"Crime type column: '{crime_type_col}'")
    print(f"Total unique crime types: {df[crime_type_col].nunique()}")
    
    print("\n\nTOP 10 CRIME TYPES:")
    print("-" * 80)
    top_crimes = df[crime_type_col].value_counts().head(10)
    
    for rank, (crime, count) in enumerate(top_crimes.items(), 1):
        pct = count / len(df) * 100
        bar_length = int(pct * 2)  # Scale bar
        bar = "█" * bar_length
        print(f"{rank:2d}. {crime:35s} {count:8,} ({pct:5.2f}%) {bar}")
    
    print(f"\nOther types: {df[crime_type_col].nunique() - 10}")
else:
    print("✗ No crime type column detected")

# 6. TEMPORAL PATTERNS

In [ ]:
print("\n\n[6/7] Temporal patterns...")
print("-" * 80)

if 'hour' in df.columns:
    print("\nINCIDENTS BY HOUR OF DAY:")
    hourly = df['hour'].value_counts().sort_index()
    max_count = hourly.max()
    
    for hour in range(24):
        count = hourly.get(hour, 0)
        bar_length = int((count / max_count) * 40) if max_count > 0 else 0
        bar = "█" * bar_length
        print(f"  {hour:02d}:00 - {count:6,} {bar}")

if 'dayofweek' in df.columns:
    print("\n\nINCIDENTS BY DAY OF WEEK:")
    days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    weekly = df['dayofweek'].value_counts().sort_index()
    max_count = weekly.max()
    
    for day_num in range(7):
        count = weekly.get(day_num, 0)
        bar_length = int((count / max_count) * 40) if max_count > 0 else 0
        bar = "█" * bar_length
        print(f"  {days[day_num]:10s} - {count:6,} {bar}")

if 'month' in df.columns:
    print("\n\nINCIDENTS BY MONTH:")
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    monthly = df['month'].value_counts().sort_index()
    max_count = monthly.max()
    
    for month_num in range(1, 13):
        count = monthly.get(month_num, 0)
        bar_length = int((count / max_count) * 40) if max_count > 0 else 0
        bar = "█" * bar_length
        print(f"  {months[month_num-1]} - {count:6,} {bar}")

# 7. SAMPLE DATA

In [ ]:
print("\n\n[7/7] Sample records...")
print("-" * 80)
print("\nFirst 3 rows of data:")
print(df.head(3).to_string())

# SUMMARY REPORT

In [ ]:
print("\n\n" + "=" * 80)
print(" " * 30 + "SUMMARY REPORT")
print("=" * 80)

summary = f"""
Dataset: {os.path.basename(DATA_PATH)}
Total records analyzed: {len(df):,}

KEY COLUMNS:
  • Date/Time: {date_col if date_col else '❌ NOT FOUND'}
  • Latitude:  {lat_col if lat_col else '❌ NOT FOUND'}
  • Longitude: {lon_col if lon_col else '❌ NOT FOUND'}
  • Crime Type: {crime_type_col if crime_type_col else '❌ NOT FOUND'}

DATA QUALITY:
  • Valid coordinates: {len(valid_coords) if lat_col and lon_col else 'N/A':,} ({coord_coverage if lat_col and lon_col else 0:.1f}%)
  • Date range: {valid_dates.min().date() if date_col and len(valid_dates) > 0 else 'N/A'} to {valid_dates.max().date() if date_col and len(valid_dates) > 0 else 'N/A'}
  • Unique crime types: {df[crime_type_col].nunique() if crime_type_col else 'N/A'}

TOP 3 CRIMES:
"""

if crime_type_col:
    for i, (crime, count) in enumerate(df[crime_type_col].value_counts().head(3).items(), 1):
        summary += f"  {i}. {crime} ({count:,} incidents)\n"

print(summary)

print("=" * 80)
print("✓ DIAGNOSTIC COMPLETE!")
print("=" * 80)

print("CONFIG = {")
print(f"    'date_column': '{date_col}',")
print(f"    'latitude_column': '{lat_col}',")
print(f"    'longitude_column': '{lon_col}',")
print(f"    'crime_type_column': '{crime_type_col}',")
print("}")
print("\nShare this output with me to proceed! 🚀")
